# 1. Set up imports

In [7]:
import tensorflow as tf
import numpy as np
import os
import pickle
print(tf.__version__)

2.10.0


# 2. Set up loss functions

In [4]:
# def calc_recon_loss(y_target, y_pred):
#     error = y_target - y_pred
#     recon_loss = np.mean(np.sum(error), axis=[1])

# 3. Dummy Encoder

In [5]:
encoder = tf.keras.models.Sequential(name="Encoder")
encoder.add(tf.keras.layers.Conv2D(16, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu', input_shape=(1,256,1)))
encoder.add(tf.keras.layers.Conv2D(32, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu'))
encoder.add(tf.keras.layers.Reshape((1,-1)))
encoder.add(tf.keras.layers.Dense(16))

encoder.build()
encoder.summary()

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 128, 16)        80        
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 64, 32)         2080      
                                                                 
 reshape (Reshape)           (None, 1, 2048)           0         
                                                                 
 dense (Dense)               (None, 1, 16)             32784     
                                                                 
Total params: 34,944
Trainable params: 34,944
Non-trainable params: 0
_________________________________________________________________


2023-02-20 15:16:45.349139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-20 15:16:45.349349: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 4. Dummy Decoder

In [6]:
decoder = tf.keras.models.Sequential(name="Decoder")
decoder.add(tf.keras.layers.Dense(128, input_shape=(1,16,1)))
decoder.add(tf.keras.layers.Reshape((1,64,-1)))
decoder.add(tf.keras.layers.Conv2DTranspose(16, kernel_size=(4,1),strides=(1,2), padding='same', activation='relu'))
decoder.add(tf.keras.layers.Conv2DTranspose(1, kernel_size=(4,1),strides=(1,2), padding='same', activation='sigmoid'))
# decoder.add(tf.keras.layers.Dense(16*128, input_shape=(1,16,1)))


decoder.build()
decoder.summary()

Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1, 16, 128)        256       
                                                                 
 reshape_1 (Reshape)         (None, 1, 64, 32)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 1, 128, 16)       2064      
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 1, 256, 1)        65        
 ranspose)                                                       
                                                                 
Total params: 2,385
Trainable params: 2,385
Non-trainable params: 0
_________________________________________________________________


# 5. Variational Autoencoder

In [11]:
class VAE:
    def __init__(self):
        # Initialize the input shape
        self.input_shape = (1,256,1)

        # Initialize the encoder
        self.encoder = tf.keras.models.Sequential(name="Encoder")
        self.encoder.add(tf.keras.layers.Conv2D(16, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu', input_shape=self.input_shape))
        self.encoder.add(tf.keras.layers.Conv2D(32, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu'))
        self.encoder.add(tf.keras.layers.Reshape((1,-1)))
        self.encoder.add(tf.keras.layers.Dense(16))

        # Initialize the decoder
        self.decoder = tf.keras.models.Sequential(name="Decoder")
        self.decoder.add(tf.keras.layers.Dense(128, input_shape=(1,16,1)))
        self.decoder.add(tf.keras.layers.Reshape((1,64,-1)))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(16, kernel_size=(4,1),strides=(1,2), padding='same', activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(1, kernel_size=(4,1),strides=(1,2), padding='same', activation='sigmoid'))

        # Intialize VAE model
        model_input = tf.keras.layers.Input(shape=self.input_shape, name="encoder_input")
        model_output = self.decoder(self.encoder(model_input))
        self.model = tf.keras.models.Model(model_input, model_output, name="Autoencoder")

    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()




In [13]:
vae = VAE()
vae.summary()

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 1, 128, 16)        80        
                                                                 
 conv2d_7 (Conv2D)           (None, 1, 64, 32)         2080      
                                                                 
 reshape_6 (Reshape)         (None, 1, 2048)           0         
                                                                 
 dense_6 (Dense)             (None, 1, 16)             32784     
                                                                 
Total params: 34,944
Trainable params: 34,944
Non-trainable params: 0
_________________________________________________________________
Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 1,

# 6. Train the Variational Autoencoder

# 7. Export the Decoder

In [125]:
decoder.save('./keras_decoder_untrained.h5')

In [ ]:
# def __init__(self):
#         super(Decoder, self).__init__()
#         c = capacity
#         self.fc = nn.Linear(in_features=latent_dims, out_features=c*wave_sized2)
#         self.conv2 = nn.ConvTranspose1d(in_channels=c*2, out_channels=c, kernel_size=4, stride=2, padding=1)
#         self.conv1 = nn.ConvTranspose1d(in_channels=c, out_channels=1, kernel_size=4, stride=2, padding=1)
            
#     def forward(self, x):
#         x = self.fc(x)
#         x = x.view(x.size(0), capacity*2, wave_sized4) # unflatten batch of feature vectors to a batch of multi-channel feature maps
# #        x = x.view(1, 32, 64) #### need to hardcode the shape in view to satisfy loading in onnx.js !!! ####
#         x = F.relu(self.conv2(x))
#         x = torch.sigmoid(self.conv1(x)) # last layer before output is sigmoid, since we are using BCE as reconstruction loss
#         return x

In [124]:
example_input = np.random.randn(1,1,256)
example_input.shape
en_pred = encoder.predict(example_input)
new_pred = decoder.predict(en_pred)
display(en_pred, en_pred.shape)

2023-02-15 16:02:47.461506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 389ms/step


2023-02-15 16:02:47.888015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 801ms/step


array([[[-0.04927252,  0.00486319,  0.01706034, -0.09054629,
         -0.08896114, -0.00797448,  0.01536477, -0.04613001,
          0.06059843, -0.00744831, -0.06729551,  0.10568736,
         -0.05514535, -0.02591524, -0.05836235,  0.02622267]]],
      dtype=float32)

(1, 1, 16)